## osumapper #4: New Map Reader


Set the input file string to a timed (having the right BPM/offset) .osu file.

It converts the map/music to Python readable format.

In [1]:
from act_newmap_prep import *

# input file here! (don't remove the "r" before string)
file_path = r'D:\osu!\Songs\1140521 ave;new feat Saori Sakura - _3Love_Chuki!!\ave;new feat. Saori Sakura - 3LoveChuki!! (Musty) [a].osu'

# Or use auto timing with music file only!!

# from act_timing import *;
# music_path = r"D:\download\audio_file.mp3"
# file_path = get_timed_osu_file(music_path);

step4_read_new_map(file_path);

## osumapper #5: Rhythm Predictor

Calculates a map's rhythm based on the music and timing.

Parameters:

"note_density" determines how many notes will be placed on the timeline, ranges from 0 to 1.<br>
"slider_favor" determines how the model favors sliders against circles, ranges from -1 to 1.<br>
"dist_multiplier" determines the distance snap. ranges from 0 to +∞. Of course 0/+∞ are not advisable.<br>
"divisor_favor" determines how the model favors notes to be on X divisors starting from a beat (white, blue, red, blue), ranges from -1 to 1 each.<br>
"slider_max_ticks" determines the max amount of time a slider can slide, ranges from 1 to +∞.

In [1]:
from act_rhythm_calc import *

model = step5_load_model();
npz = step5_load_npz();
params = step5_set_params(dist_multiplier=1, note_density=0.32, slider_favor=0, divisor_favor=[0] * 4, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);

368 notes predicted.


In [2]:
step5_save_predictions(converted);

## osumapper #6: Map flow generator

Generate the final map using a Generative Adversarial Network.

Parameters:

- note_distance_basis: the baseline for distance snap between notes
- max_ticks_for_ds: max number of time ticks (each 1/4) that it uses the distance snap
- next_from_slider_end: use slider end instead of slider head for calculating distance
- I haven't tested the others 

In [7]:
from act_gan import *;

GAN_PARAMS = {
    "divisor" : 4,
    "good_epoch" : 6,
    "max_epoch" : 25,
    "note_group_size" : 10,
    "g_epochs" : 7,
    "c_epochs" : 3,
    "g_batch" : 50,
    "g_input_size" : 50,
    "c_true_batch" : 50,
    "c_false_batch" : 10,
    "note_distance_basis" : 200,
    "next_from_slider_end" : False,
    "max_ticks_for_ds" : 2
};

step6_set_gan_params(GAN_PARAMS);
osu_a, data = step6_run_all();

# of groups: 33
Group 0, Epoch 1: G loss: 0.24790037316935404 vs. C loss: 0.2262388070424398
Group 0, Epoch 2: G loss: 0.28358536320073263 vs. C loss: 0.16672679781913757
Group 0, Epoch 3: G loss: 0.26922269804137094 vs. C loss: 0.20779886841773987
Group 0, Epoch 4: G loss: 0.0838413078870092 vs. C loss: 0.20252553621927896
Group 0, Epoch 5: G loss: 0.04425418110830443 vs. C loss: 0.18909139931201935
Group 0, Epoch 6: G loss: 0.10191006000552859 vs. C loss: 0.1783087452252706
Group 1, Epoch 1: G loss: 0.2881387876612799 vs. C loss: 0.26865369578202564
Group 1, Epoch 2: G loss: 0.22215533469404494 vs. C loss: 0.16718232134977976
Group 1, Epoch 3: G loss: 0.29192224570683073 vs. C loss: 0.12480976184209187
Group 1, Epoch 4: G loss: 0.3819560046706881 vs. C loss: 0.17794767022132874
Group 1, Epoch 5: G loss: 0.15578151920012065 vs. C loss: 0.20804613331953684
Group 1, Epoch 6: G loss: 0.05712507452283587 vs. C loss: 0.18427309393882751
Group 2, Epoch 1: G loss: 0.2838212102651596 vs. C lo

Group 17, Epoch 3: G loss: 0.20213674860341208 vs. C loss: 0.16006428003311157
Group 17, Epoch 4: G loss: 0.3147542966263635 vs. C loss: 0.1617212196191152
Group 17, Epoch 5: G loss: 0.2527074047497341 vs. C loss: 0.18972897032896677
Group 17, Epoch 6: G loss: 0.05785740752305303 vs. C loss: 0.19285001357396445
Group 18, Epoch 1: G loss: 0.29194206637995584 vs. C loss: 0.26264072954654694
Group 18, Epoch 2: G loss: 0.18255602887698583 vs. C loss: 0.17617269357045492
Group 18, Epoch 3: G loss: 0.17933100249086106 vs. C loss: 0.16321403781572977
Group 18, Epoch 4: G loss: 0.3744364380836487 vs. C loss: 0.16807648042837778
Group 18, Epoch 5: G loss: 0.12315477217946734 vs. C loss: 0.20175573726495108
Group 18, Epoch 6: G loss: 0.03942712476210935 vs. C loss: 0.2001135398944219
Group 19, Epoch 1: G loss: 0.3450791473899569 vs. C loss: 0.2548838754494985
Group 19, Epoch 2: G loss: 0.24351337552070618 vs. C loss: 0.15675740440686545
Group 19, Epoch 3: G loss: 0.38224040610449656 vs. C loss: 

### Since the generation will take a while...

we can appreciate a nice picture of Cute Sophie!!

<img src="https://i.imgur.com/Ko2wogO.jpg" />

After the generation is finished, save the data into an .osu file!

In [8]:
from act_final import *

saved_osu_name = step7_save_osu_file(osu_a, data);

success! file converted to json.
elapsed time: 0.117 s

finished on: 2020-10-16 16:23:53.721972


If it works alright, you should have a nice .osu file under the folder of these notebooks now!

If it does not work, please tell me the problem so probably I could fix it!

@2020/10/16